In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pennylane as qml
from tqdm import tqdm
from scipy.stats import ttest_1samp
from scipy.stats import combine_pvalues
import pandas as pd

from device_var_lse_solver import DeviceVarLSESolver
from device import Device
from device import DeviceType
from qiskit_aer import AerSimulator

from sparse_cond_matrix import build_pauli_sum_A_strings, build_matrix_from_paulis
from non_dynamical_ansatz import fixed_layered_ansatz


qml.QubitStateVector = qml.StatePrep

"""
To run this script you need the following packages:
- PennyLane (but the latest version does not have the QubitStateVector class (which is needed), so we redefine it above)
- Numpy < 2: need to install something like numpy 1.26.4
- Python 3.12
- torch 2.2.2
"""

depth = 10

# As defined by [10]
def TRC_ADA(iteration_count : dict):
    trc = 0
    for d in iteration_count:
        z = iteration_count.get(d)
        trc += d*z
    return trc

def TRC_ASA(iteration_count : dict):
    trc = 0
    for d in iteration_count:
        z = iteration_count.get(d)
        trc += z*depth
    return trc

In [ ]:
# Testing we get the correct condition number for the matrix A
for KAPPA in [1.01,10,400]:
    pauli_strings, coeffs, b_state = build_pauli_sum_A_strings(qubits=3, J = 0.1, kappa=KAPPA)
    A_mat = build_matrix_from_paulis(pauli_strings, coeffs)

    cond = np.linalg.cond(A_mat)
    print(f"alpha = {KAPPA:.4e} -> condition number = {cond:.4e}")

b = np.ones(2**3)
b = b / np.linalg.norm(b)

## Below we define condition numbers (kappa) and other parameters

In [ ]:
# Nr of qubits
qubits = 3
n = 2 ** qubits

# Condition numbers 
kappa_list = [20, 30, 40, 70, 100]

# Device for noise simulation
device_noise = Device(DeviceType.QISKIT_AER, qubits=qubits)

# Save results?
save = True

# Number of runs
redo_calc = 10


# Hyperparameters
lr = 0.01
threshold = 0.0001
steps = 5000
epochs = 10
abort = 200

#Lists for saving results
results_list_dynamic = np.zeros([redo_calc, n])
results_list_static = np.zeros([redo_calc, n])
results_list_dynamic_noise = np.zeros([redo_calc, n])
results_list_static_noise = np.zeros([redo_calc, n])

TRC_list_dynamic = np.zeros(redo_calc)
TRC_list_static = np.zeros(redo_calc)
TRC_list_dynamic_noise = np.zeros(redo_calc)
TRC_list_static_noise = np.zeros(redo_calc)


# Needed for the static ansatz
depth = 10
param_shape = (qubits + depth*(qubits+qubits -2),)


# Running code for different condition numbers
for KAPPA in kappa_list:
    for i in range(redo_calc):
        print(f"Run nr. {i+1}/{redo_calc}")
        A, coeffs, b = build_pauli_sum_A_strings(qubits=qubits, J=0.1, kappa=KAPPA)

        ###################################
        #####    DYNAMIC NOISELESS    #####
        ###################################

        lse_dyn_dynamic = DeviceVarLSESolver(A, 
                    b, 
                    coeffs=coeffs,
                    method="hadamard", 
                    local=True, 
                    lr=lr, 
                    threshold=threshold,
                    steps=steps,
                    epochs=epochs, 
                    abort=abort,) 

        solution_local, param_local, it_count_local = lse_dyn_dynamic.solve()
        local_dense_trc = TRC_ADA(it_count_local)
        results_list_dynamic[i, :] = solution_local
        TRC_list_dynamic[i] = local_dense_trc


          
        ###################################
        #####    STATIC  NOISELESS    #####
        ###################################

        lse_stat_static = DeviceVarLSESolver(A,
                    b, 
                    coeffs=coeffs,
                    method='hadamard',
                    ansatz=fixed_layered_ansatz,
                    weights=param_shape,
                    local=True, 
                    lr=lr, 
                    steps=steps,
                    threshold=threshold, 
                    epochs=epochs,
                    abort=abort,)
            

        solution_local, param_local, it_count_local = lse_stat_static.solve()
        results_list_static[i, :] = solution_local
        TRC_list_static[i] = TRC_ASA(it_count_local)


        ###################################
        #####    DYNAMIC NOISE    #####
        ###################################
        lse_dyn_dynamic_noise = DeviceVarLSESolver(A, 
                    b, 
                    coeffs=coeffs,
                    method="hadamard", 
                    local=True, 
                    lr=lr, 
                    threshold=threshold,
                    steps=steps,
                    epochs=epochs, 
                    device=device_noise, 
                    abort=abort,)
        solution_local, param_local, it_count_local = lse_dyn_dynamic_noise.solve()
        local_dense_trc = TRC_ADA(it_count_local)
        results_list_dynamic_noise[i, :] = solution_local
        TRC_list_dynamic_noise[i] = local_dense_trc

        ###################################
        #####    STATIC  NOISE    #####
        ###################################
        lse_stat_static_noise = DeviceVarLSESolver(A,
                    b, 
                    coeffs=coeffs,
                    method='hadamard',
                    ansatz=fixed_layered_ansatz,
                    weights=param_shape,
                    local=True, 
                    lr=lr, 
                    steps=steps,
                    threshold=threshold, 
                    epochs=epochs,
                    device=device_noise,
                    abort=abort,)
        solution_local, param_local, it_count_local = lse_stat_static_noise.solve()
        results_list_static_noise[i, :] = solution_local
        TRC_list_static_noise[i] = TRC_ASA(it_count_local)


        

# Save results
    if save == True:
        filename = f"data/condition_nr/result_local_condnr_{KAPPA}_w_noise.npz"
        save_dict = {
            'A': A,
            'coeffs': coeffs,
            'b': b,
            'results_list_dynamic': results_list_dynamic,
            'TRC_list_dynamic': TRC_list_dynamic,
            'results_list_static': results_list_static,
            'TRC_list_static': TRC_list_static,
            'results_list_dynamic_noise': results_list_dynamic_noise,
            'TRC_list_dynamic_noise': TRC_list_dynamic_noise,
            'results_list_static_noise': results_list_static_noise,
            'TRC_list_static_noise': TRC_list_static_noise,
            'nr_runs': redo_calc,
            'KAPPA': KAPPA,
            'qubits': qubits,

        }
        np.savez(filename, **save_dict)
        print(f"Saved {filename}")


### To plot the current saved data use/uncomment the kappa_list below

In [ ]:
# # Nr of qubits
# qubits = 3
# n = 2 ** qubits

# # # What we have saved data for:
# kappa_list = [1.04e+00,1.49e+00, 10, 20, 30, 40, 50, 70, 100, 1.58e+02]

In [ ]:
# Lists for saving results
mean_results_dynamic = np.zeros([len(kappa_list), n])
std_results_dynamic = np.zeros([len(kappa_list), n])
mean_trc_dynamic = np.zeros(len(kappa_list))
std_trc_dynamic = np.zeros(len(kappa_list))

mean_results_static = np.zeros([len(kappa_list), n])
std_results_static = np.zeros([len(kappa_list), n])
mean_trc_static = np.zeros(len(kappa_list))
std_trc_static = np.zeros(len(kappa_list))

mean_results_dynamic_noise = np.zeros([len(kappa_list), n])
std_results_dynamic_noise = np.zeros([len(kappa_list), n])
mean_trc_dynamic_noise = np.zeros(len(kappa_list))
std_trc_dynamic_noise = np.zeros(len(kappa_list))

mean_results_static_noise = np.zeros([len(kappa_list), n])
std_results_static_noise = np.zeros([len(kappa_list), n])
mean_trc_static_noise = np.zeros(len(kappa_list))
std_trc_static_noise = np.zeros(len(kappa_list))

# Lists for t-test results
ttest_list_dynamic_p = np.zeros([len(kappa_list)])
ttest_list_static_p = np.zeros([len(kappa_list)])
ttest_list_dynamic_noise_p = np.zeros([len(kappa_list)])
ttest_list_static_noise_p = np.zeros([len(kappa_list)])

ttest_list_dynamic_p_list = np.zeros([len(kappa_list), n])
ttest_list_static_p_list = np.zeros([len(kappa_list), n])
ttest_list_dynamic_noise_p_list = np.zeros([len(kappa_list), n])
ttest_list_static_noise_p_list = np.zeros([len(kappa_list), n])

ttest_list_dynamic_stat = np.zeros([len(kappa_list), n])
ttest_list_static_stat = np.zeros([len(kappa_list), n])
ttest_list_dynamic_noise_stat = np.zeros([len(kappa_list), n])
ttest_list_static_noise_stat = np.zeros([len(kappa_list), n])

# List for classical solution
classical_solution = np.zeros([len(kappa_list), n])

# Load saved results for different condition numbers
for i, KAPPA in enumerate(kappa_list):

    filename = f"data/condition_nr/result_local_condnr_{KAPPA}_w_noise.npz"
    data = np.load(filename)
    
    A = data['A']
    coeffs = data['coeffs']
    b = data['b']
    results_list_dynamic = data['results_list_dynamic']
    TRC_list_dynamic = data['TRC_list_dynamic']
    results_list_static = data['results_list_static']
    TRC_list_static = data['TRC_list_static']
    results_list_dynamic_noise = data['results_list_dynamic_noise']
    TRC_list_dynamic_noise = data['TRC_list_dynamic_noise']
    results_list_static_noise = data['results_list_static_noise']
    TRC_list_static_noise = data['TRC_list_static_noise']
    nr_runs = data['nr_runs']
    KAPPA = data['KAPPA']
    qubits = data['qubits'] 


    mean_results_dynamic[i, :] = np.mean(results_list_dynamic, axis=0)
    std_results_dynamic[i, :] = np.std(results_list_dynamic, axis=0)/ np.sqrt(nr_runs)
    mean_trc_dynamic[i] = np.mean(TRC_list_dynamic)
    std_trc_dynamic[i] = np.std(TRC_list_dynamic)/ np.sqrt(nr_runs)

    mean_results_static[i, :] = np.mean(results_list_static, axis=0)
    std_results_static[i, :] = np.std(results_list_static, axis=0)/ np.sqrt(nr_runs)
    mean_trc_static[i] = np.mean(TRC_list_static)
    std_trc_static[i] = np.std(TRC_list_static)/ np.sqrt(nr_runs)

    mean_results_dynamic_noise[i, :] = np.mean(results_list_dynamic_noise, axis=0)  
    std_results_dynamic_noise[i, :] = np.std(results_list_dynamic_noise, axis=0)/ np.sqrt(nr_runs)
    mean_trc_dynamic_noise[i] = np.mean(TRC_list_dynamic_noise)
    std_trc_dynamic_noise[i] = np.std(TRC_list_dynamic_noise)/ np.sqrt(nr_runs)

    mean_results_static_noise[i, :] = np.mean(results_list_static_noise, axis=0)
    std_results_static_noise[i, :] = np.std(results_list_static_noise, axis=0)/ np.sqrt(nr_runs)
    mean_trc_static_noise[i] = np.mean(TRC_list_static_noise)
    std_trc_static_noise[i] = np.std(TRC_list_static_noise)/ np.sqrt(nr_runs)

    # compute normalized classical solution for comparison, therefore first re-compose system matrix A
    pauli_strings, coeffs, b = build_pauli_sum_A_strings(qubits=int(qubits), kappa=KAPPA)
    A_mat = build_matrix_from_paulis(pauli_strings, coeffs)
    x_classical = np.linalg.solve(A_mat, b)
    normalized_classical_solution = np.square(np.abs(x_classical / np.linalg.norm(x_classical)))

    classical_solution[i, :] = normalized_classical_solution

    # compute t-test
    dyn_stat, dyn_p = ttest_1samp(results_list_dynamic, normalized_classical_solution)
    ttest_list_dynamic_stat[i, :] = np.abs(dyn_stat)
    ttest_list_dynamic_p[i] = combine_pvalues(dyn_p)[1]
    ttest_list_dynamic_p_list[i, :] = dyn_p
    
    
    stat_stat, stat_p = ttest_1samp(results_list_static, normalized_classical_solution)
    ttest_list_static_stat[i, :] = np.abs(stat_stat)
    ttest_list_static_p[i] = combine_pvalues(stat_p)[1]
    ttest_list_static_p_list[i, :] = stat_p
    
  
    dyn_noise_stat, dyn_noise_p = ttest_1samp(results_list_dynamic_noise, normalized_classical_solution)
    ttest_list_dynamic_noise_stat[i, :] = np.abs(dyn_noise_stat)
    ttest_list_dynamic_noise_p[i] = combine_pvalues(dyn_noise_p)[1]
    ttest_list_dynamic_noise_p_list[i, :] = dyn_noise_p
    
    
    
    stat_noise_stat, stat_noise_p = ttest_1samp(results_list_static_noise, normalized_classical_solution)
    ttest_list_static_noise_stat[i, :] = np.abs(stat_noise_stat)
    ttest_list_static_noise_p[i] = combine_pvalues(stat_noise_p)[1]
    ttest_list_static_noise_p_list[i, :] = stat_noise_p
    
  


In [ ]:
# Plotting results

error_list = np.zeros([len(kappa_list), 4])

for i, KAPPA in enumerate(kappa_list):

    plt.figure(figsize=(7, 4))
    plt.title(f"Results for $\\kappa$ = {KAPPA}")

    x = np.arange(len(classical_solution[i, :])) 
    width = 0.15 
    multiplier = 0
    
    plt.bar(x, classical_solution[i, :], width = width, label='Ground truth')
    plt.bar(x + width,  mean_results_dynamic[i, :], yerr=std_results_dynamic[i, :], width=width, capsize=3, label='Dynamic')
    plt.bar(x + 2*width, mean_results_static[i, :], yerr=std_results_static[i, :], width=width, capsize=3, label='Static')
    plt.bar(x + 3*width, mean_results_dynamic_noise[i, :], yerr=std_results_dynamic_noise[i, :], width=width, capsize=3, label='Dynamic (noisy)')
    plt.bar(x + 4*width, mean_results_static_noise[i, :], yerr=std_results_static_noise[i, :], width=width, capsize=3, label='Static (noisy)')
    
    error_dynamic = np.linalg.norm(mean_results_dynamic[i, :] - np.abs(classical_solution[i, :])) / np.linalg.norm(classical_solution[i, :])
    error_static = np.linalg.norm(mean_results_static[i, :] - np.abs(classical_solution[i, :])) / np.linalg.norm(classical_solution[i, :])
    error_dynamic_noise = np.linalg.norm(mean_results_dynamic_noise[i, :] - np.abs(classical_solution[i, :])) / np.linalg.norm(classical_solution[i, :])
    error_static_noise = np.linalg.norm(mean_results_static_noise[i, :] - np.abs(classical_solution[i, :])) / np.linalg.norm(classical_solution[i, :])

    error_list[i, 0] = error_dynamic
    error_list[i, 1] = error_static
    error_list[i, 2] = error_dynamic_noise
    error_list[i, 3] = error_static_noise


    plt.legend()
    plt.show()

In [ ]:
# Plotting relative error

plt.figure(figsize=(6, 4))
plt.plot(kappa_list, error_list[:, 0], label='Dynamic', color = 'dodgerblue')
plt.plot(kappa_list, error_list[:, 1], label='Static', color = 'darkorange')
plt.plot(kappa_list, error_list[:, 2], label='Dynamic (noisy)', color = 'dodgerblue', linestyle='--')
plt.plot(kappa_list, error_list[:, 3], label='Static (noisy)', color = 'darkorange', linestyle='--')


plt.xlabel('Kappa')
plt.ylabel('Relative error')
plt.legend()

plt.scatter(kappa_list[-7], error_list[:, 0][-7], color='red', marker='X', zorder=5)
plt.scatter(kappa_list[-7], error_list[:, 1][-7], color='red', marker='X', zorder=5)
plt.scatter(kappa_list[-7], error_list[:, 2][-7], color='red', marker='X', zorder=5)
plt.scatter(kappa_list[-7], error_list[:, 3][-7], color='red', marker='X', zorder=5)
plt.savefig('relative_error_w_crosses20.png')
plt.show()

In [ ]:
# Plotting t-test results
plt.figure(figsize=(6, 4))
plt.plot(kappa_list, ttest_list_static_stat.mean(axis=1), label='Static ', color = 'darkorange')
#plt.plot(kappa_list, ttest_list_dynamic_stat.mean(axis=1), label='Dynamic ', color = 'dodgerblue')
plt.plot(kappa_list, ttest_list_static_noise_stat.mean(axis=1), label='Static (noisy)', color = 'darkorange', linestyle='--')
plt.plot(kappa_list, ttest_list_dynamic_noise_stat.mean(axis=1), label='Dynamic (noisy)', color = 'dodgerblue', linestyle='--')

plt.xlabel('$\\kappa$')
plt.ylabel('T-test statistic')
plt.legend()

#plt.savefig('ttest_wo_static.png')

In [ ]:
# Making a dataframe for the p values, not taking the mean

pd.reset_option('display.float_format')
pd.set_option('display.float_format', '{:.6e}'.format)
df = pd.DataFrame({
    'kappa': kappa_list,
    'Dynamic': ttest_list_dynamic_p,
    'Static': ttest_list_static_p,
    'Dynamic (noisy)': ttest_list_dynamic_noise_p,
    'Static (noisy)': ttest_list_static_noise_p
})

df

In [ ]:
# Plotting TRC

plt.plot(kappa_list, mean_trc_dynamic, label='Dynamic ansatz', marker='o')
plt.fill_between(kappa_list, mean_trc_dynamic - std_trc_dynamic, mean_trc_dynamic + std_trc_dynamic, alpha=0.2)
plt.plot(kappa_list, mean_trc_static, label='Static ansatz', marker='o')
plt.fill_between(kappa_list, mean_trc_static - std_trc_static, mean_trc_static + std_trc_static, alpha=0.2)
plt.plot(kappa_list, mean_trc_dynamic_noise, label='Dynamic ansatz with noise', marker='o')
plt.fill_between(kappa_list, mean_trc_dynamic_noise - std_trc_dynamic_noise, mean_trc_dynamic_noise + std_trc_dynamic_noise, alpha=0.2)
plt.plot(kappa_list, mean_trc_static_noise, label='Static ansatz with noise', marker='o')
plt.fill_between(kappa_list, mean_trc_static_noise - std_trc_static_noise, mean_trc_static_noise + std_trc_static_noise, alpha=0.2)
plt.xlabel('Kappa')
plt.ylabel('Trace of the number of iterations')

plt.title('Trace of the number of iterations for different kappa values')
plt.legend()
plt.show()